# CODE QUI DOIT FONCTIONNER

In [63]:
import os
import pandas as pd
import networkx as nx
from pyvis.network import Network
from IPython.display import display, HTML
import ast
import json
import numpy as np
import glob

In [34]:
dossier = '../data/archive/result'
hierarchical_topics = pd.read_csv(f"{dossier}/database_hierarchical_topics.csv",sep=',',encoding='utf8')
news_data = pd.read_csv(f"{dossier}/database_update.csv",sep=',',encoding='utf8')
all_topic = pd.read_csv(f"{dossier}/all_topics.csv",sep=',',encoding='utf8').iloc[1:].reset_index(drop=True)

In [35]:
print(all_topic.head())

   Topic  Count                               Name  \
0      0    741  0_obamacare_insurance_health_care   
1      1    557   1_emails_server_email_classified   
2      2    546    2_gun_guns_awrhawkins_amendment   
3      3    458    3_campus_milo_students_berkeley   
4      4    447    4_poll_percent_voters_electoral   

                                      Representation  \
0  ['obamacare', 'insurance', 'health', 'care', '...   
1  ['emails', 'server', 'email', 'classified', 'f...   
2  ['gun', 'guns', 'awrhawkins', 'amendment', 'ha...   
3  ['campus', 'milo', 'students', 'berkeley', 'un...   
4  ['poll', 'percent', 'voters', 'electoral', 'po...   

                                 Representative_Docs  
0  ['WASHINGTON — Vice Mike Pence top Republicans...  
1  ['Following prepared text remarks Director Jam...  
2  ['people armed , Donald J. Trump says rallies ...  
3  ['BERKELEY , Calif. — Fires burned cradle free...  
4  ['Washington ( CNN ) Democratic nominee Hillar...  


In [80]:
news_net = Network(height="750px", width="100%", bgcolor="#222222", font_color="white", select_menu=True, layout=False)    
news_net.barnes_hut()
max_original_topic = hierarchical_topics.Parent_ID.astype(int).min() - 1

        # Extract mapping from ID to name
topic_to_name = dict(zip(hierarchical_topics.Child_Left_ID, hierarchical_topics.Child_Left_Name))
topic_to_name.update(dict(zip(hierarchical_topics.Child_Right_ID, hierarchical_topics.Child_Right_Name)))
topic_to_name.update(dict(zip(hierarchical_topics.Parent_ID, hierarchical_topics.Parent_Name)))

topic_to_name = {topic: name for topic, name in topic_to_name.items()}
#print((topic_to_name))
        # Create tree
tree = {row[1].Parent_ID: [row[1].Child_Left_ID, row[1].Child_Right_ID]
        for row in hierarchical_topics.iterrows()}


for parent,childs in tree.items():
        src = parent
        src_names = topic_to_name[parent]
        lvl = hierarchical_topics.loc[hierarchical_topics['Parent_ID'] == parent, 'Distance'].values[0]
        news_net.add_node(src, src_names, title=src,level=src)
        for child in childs:
                dst = child
                dst_names = topic_to_name[child]
                news_net.add_node(dst, dst_names, title=dst,level=dst)
                news_net.add_edge(src, dst,values=lvl)
        topic_str = hierarchical_topics.loc[hierarchical_topics['Parent_ID'] == parent, 'Topics'].values[0]
        topics = set(int(topic.strip()) for topic in topic_str.strip('[]').split(','))
        #print(f"topics : {topics}")
        filter_child = topics - set(childs)
        for child in filter_child:
                childs_names = all_topic.loc[all_topic['Topic'] == child, 'Name'].values[0]
                #print(child, childs_names)
                news_net.add_node(child, str(childs_names), title=str(child))
                news_net.add_edge(src, child,values=lvl)
           
"""
for index, row in all_topic.iterrows():
        list_doc = news_data.loc[news_data['topic'] == row['Topic'],'id', 'title',"keyword"].values
        
        for id,title,keywords in list_doc :
                news_net.add_node(id,title,title=id)
                news_net.add_edge(row['Topic'],id)
                for keyword,distance in keywords :
                        max = news_net.num_nodes()+1
                        news_net.add_nodes(max,keyword,title=max)
                        news_net.add_edge(id,max,distance)
"""
neighbor_map = news_net.get_adj_list()
for node in news_net.nodes:
# Assurez-vous que node["title"] est une chaîne de caractères
        node_title_str = node["label"]
        neighbors_str = ", ".join(str(neighbor) for neighbor in neighbor_map[node["id"]])
        # Utilisez la version chaîne de caractères de node["title"] pour la concaténation
        node["title"] = node_title_str + "\n Neighbors : [" + neighbors_str + "]"
        node["value"] = len(neighbor_map[node["id"]])




news_net.toggle_physics(True)
news_net.show_buttons(True)
news_net.inherit_edge_colors(True)
news_net.save_graph("graphs_.html")   


# Tree test

In [44]:
max_distance = None
width = 1 
hier_topics = pd.read_csv(f"archive/test/result/database_hierarchical_topics.csv")
if max_distance is None:
    max_distance = hier_topics.Distance.max() + 1

max_original_topic = hier_topics.Parent_ID.astype(int).min() - 1

# Extract mapping from ID to name
topic_to_name = dict(zip(hier_topics.Child_Left_ID, hier_topics.Child_Left_Name))
topic_to_name.update(dict(zip(hier_topics.Child_Right_ID, hier_topics.Child_Right_Name)))
topic_to_name = {str(topic): name[:100] for topic, name in topic_to_name.items()}

# Create tree
tree = {str(row[1].Parent_ID): [str(row[1].Child_Left_ID), str(row[1].Child_Right_ID)]
        for row in hier_topics.iterrows()}

def get_tree(start, tree):
    """ Based on: https://stackoverflow.com/a/51920869/10532563 """

    def _tree(to_print, start, parent, tree, grandpa=None, indent=""):

        # Get distance between merged topics
        distance = hier_topics.loc[(hier_topics.Child_Left_ID == parent) |
                                    (hier_topics.Child_Right_ID == parent), "Distance"]
        distance = distance.values[0] if len(distance) > 0 else 10

        if parent != start:
            if grandpa is None:
                to_print += topic_to_name[parent]
            else:
                if int(parent) <= max_original_topic:
                    to_print += "■──" + topic_to_name[parent] + f" ── Topic: {parent}" + "\n"
                    
                else:
                    to_print += topic_to_name[parent] + "\n"

        if parent not in tree:
            return to_print

        for child in tree[parent][:-1]:
            to_print += indent + "├" + "─"
            to_print = _tree(to_print, start, child, tree, parent, indent + "│" + " " * width)

        child = tree[parent][-1]
        to_print += indent + "└" + "─"
        to_print = _tree(to_print, start, child, tree, parent, indent + " " * (width+1))

        return to_print

    to_print = "." + "\n"
    to_print = _tree(to_print, start, start, tree)
    return to_print

start = str(hier_topics.Parent_ID.astype(int).max())
print(get_tree(start, tree))

FileNotFoundError: [Errno 2] No such file or directory: 'archive/test/result/database_hierarchical_topics.csv'

In [6]:
import pandas as pd

df = pd.read_csv(f"../data/archive/database_merge.csv")
key_to_check = 7174
print(df[7170:7180])
print(df.index[7170:7180])
if key_to_check in df.index:
    # Proceed with operations on 
    print("Key exists!")
else:
    print("Key does not exist in the DataFrame index.")


           id                                              title  \
7170  25698.0  Without Obama, Shinzo Abe’s Approach to U.S.-J...   
7171  25700.0  Giving Up ‘Mostly Everything’ to Care for His ...   
7172  25701.0  Carrie Fisher, Trump, Israel: Your Wednesday E...   
7173  25703.0  How to Have a Dinner Party: Friends Not Requir...   
7174  25704.0                   The Fighter - The New York Times   
7175  25705.0  A Brother’s Crimes and a Sister’s Betrayal Mak...   
7176  25708.0  Uncertainty Over New Chinese Law Rattles Forei...   
7177  25709.0  A Majority Agreed She Was Raped by a Stanford ...   
7178  25710.0  Searching for the Human Factor in Deadly Avala...   
7179  25713.0  Michael Skakel’s Murder Conviction Has Been Re...   

         publication                             author        date    year  \
7170  New York Times                        Motoko Rich  2017-01-05  2017.0   
7171  New York Times                          John Otis  2016-12-29  2016.0   
7172  New York

In [6]:
from pyvis.network import Network
g = Network()
g.add_nodes([1,2,3], value=[10, 100, 400],
                         title=['I am node 1', 'node 2 here', 'and im node 3'],
                         x=[21.4, 54.2, 11.2],
                         y=[100.2, 23.54, 32.1],
                         label=['NODE 1', 'NODE 2', 'NODE 3'],
                         color=['#00ff1e', '#162347', '#dd4b39'])
g.add_node(1,"test")
g.save_graph('nx.html')